<h1>Image crawler</h1>
<b>Introduction</b><br/>
<div>In this notebook, we are going to find pictures of Anne Shirley's quotes by crawling the web, extract the text of the pictures and translate them into Persian, then paste the translated text on the new pictures. We do this process by searching the web using image crawlers. Many tools can be found for this task provided by various developers, whatever we use in this notebook also mention its source. We do this in the following steps.</div>

<div>A Web crawler, sometimes called a <b>spider</b> or <b>spiderbot</b> and often shortened to <b>crawler</b>, is an Internet bot that systematically browses the World Wide Web and that is typically operated by search engines for the purpose of Web indexing (web spidering).<a href="https://en.wikipedia.org/wiki/Web_crawler#cite_note-1">[1]</a></div><br/>
<div>The image crawler application is used to collect a multitude of images from websites. The images can be viewed as thumbnails or saved to a given folder for enhanced processing.<a href="https://sourceforge.net/projects/imagecrawler/">[2]</a><br/>
</div>

<b>Contents</b><br/>
<ul>
  <li>Installations</li>
      <ul>
          <li>icrawler</li>
          <li>easyocr</li>
          <li>pyspellchecker</li>
          <li>fuzzywuzzy</li>
          <li>deep-translator</li>
      </ul>
  <li>Import libraries</li>
  <li>Implementation helper functions</li>
  <li>Main body</li>
</ul>

<h1><b>Installations</b></h1>

<b>Install icrawler</b><br/>
In this project, we use the icrawler library, if you haven't installed it yet, you should install it(Python 3.5+ recommended).<br/>
With this package, you can write a multiple thread crawler easily by focusing on the contents you want to crawl, keeping away from troublesome problems like exception handling, thread scheduling and communication.<a href='https://pypi.org/project/icrawler/'>[3]</a>

It also provides built-in crawlers for popular image sites like Flickr and search engines such as Google, Bing and Baidu.<a href='https://pypi.org/project/icrawler/'>[4]</a>

In [2]:
# if you haven't installed it yet, you should install it with the following command.
# Run this line just first time
%pip install icrawler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h1>easyocr</h1>
EasyOCR is a Python package that provides a ready-to-use OCR engine and supports 80+ languages. EasyOCR is easy to install and very straightforward to use. This makes it a great solution for performing OCR with Python. You only need to install the PyTorch (required on Windows only) and EasyOCR packages, and then you can start extracting text from images using Python.

To use EasyOCR on Windows, you must install the PyTorch and EasyOCR packages. Run the following commands in sequence to install the packages:

In [3]:
%pip install torch torchvision torchaudio
%pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 KB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


<b>pyspellchecker</b>

In [4]:
%pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.8 MB/s eta 0:00:00


<b>fuzzywuzzy</b>

In [5]:
%pip install fuzzywuzzy
%pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.1 MB/s eta 0:00:00


<b>deep-translator</b>

In [6]:
%pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h1><b>Import libraries</b></h1>

In [3]:
import datetime
import sys
import os,shutil
import re
#Options for spell cheking
# %pip uninstall PyEnchant
# %pip install textblo
# %pip install jamspell
# %pip install autocorrect
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from icrawler.builtin import bing
from icrawler.builtin import google 
#Other options
# from icrawler.builtin import baidu
# from icrawler.builtin import flickr

from typing import Literal
#import arabic_reshaper
#from bidi.algorithm import get_display
from string import ascii_letters
import textwrap
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from deep_translator import GoogleTranslator
#import pytesseract(replaced by EasyOCR)
import easyocr


<h1>Initailize</h1>

In [4]:
workspace = ""
root_directory =""
quote_path = ""
output_path=""
wallpaper_keyword = "sunny"
anne_keyword="AnneShirleyQuotes"
# The maximum number for Crawling image from web
max_number = 35
# fuzzy text matching threshold(detected text and anne book content)
threshold = 90

translator = GoogleTranslator(source='auto', target='fa')

spell = SpellChecker()

# Replaced by easyocr tools
#pytesseract.pytesseract.tesseract_cmd =(r'/usr/bin/tesseract')
#Whe local: 'C:/Program Files/Tesseract-OCR/tesseract.exe' 

lang_list = ['en']
#You can pass several languages at once but not all languages can be used together. 
# English is compatible with every language and languages that share common 
# characters are usually compatible with each other.

# The line is for loading a model into memory. It takes some time but it needs to be run only once.
easyocr_reader = easyocr.Reader(lang_list,gpu=False)
# In case you do not have a GPU, or your GPU has low memory, you can run the model in CPU-only mode by adding gpu=False.


<h1><b>Implementation helper functions</b></h1>

Implementation of get_crawled_image_names

In [5]:
# returns crawled images directory and list of images in
def get_crawled_image_names(parent_path=None):
    # get list of crawled images
    # Note: crawled imaged by icrawler is stored in a folder named as 'Images'
    # in root directory
    list_ = os.listdir(parent_path)
    # filter files in the 'Images' folder by valid extensions
    ext = ['jpg','jpeg','png','bmp']

    # Filter file names based on given extensions in list 2
    filtered = [x for x in list_ if any(y in x for y in ext)]
    
    # Returns crawled image names as list
    return filtered


Implementation of text cleaning

In [6]:

def clean_text(text=''):
  # re.IGNORECASE ignoring cases
  # compilation step to escape the word for all cases
  compiled = re.compile(re.escape('anne'), re.IGNORECASE)
  text = compiled.sub('', text)
  compiled = re.compile(re.escape('shirley'), re.IGNORECASE)
  text = compiled.sub('', text)
  # Step 1: replace linebreak characters by white space
  text = text.replace('\n',' ')
  
  # Step 2: Replace extra white spaces by single white space
  spaces = re.compile('\s+')
  text = re.sub(spaces, ' ', text)
  #Step 3: Unnecessary characters Removal
  text = re.sub(r"[^a-zA-Z0-9']", " ", text) 
    
  if text ==' ': text =''

  return text

Implementation of fuzzy search

In [7]:
def fuzzy_match(str1="",book_path=None):
    best_file=""
    best_match = 0
    list_ = os.listdir(book_path)
    for f in list_:
        path_ = book_path +"/"+ str(f) 
        file = open(path_,"r")

        content = file.read()
        ratio = fuzz.token_set_ratio(str1, content)
        if best_match < ratio:
           best_match = ratio
           best_file = f

    return best_match, best_file
    

Implementation of spell_check

In [8]:

def spell_check(text=""):
    words = text.split()
    final_text=""
    for word in words:
        misspelled = spell.unknown([word])
        if len(misspelled)==1:
           correct = spell.correction(word)
           if correct == None:
              final_text = final_text+ " "+ word
           else:
              final_text = final_text+" "+ spell.correction(word)
        else:
            # Ignore from first empty space
            if final_text =="":
               final_text = word
            else:
               final_text = final_text+ " "+ word
    
    return final_text


Implementaion toggle_safe_mode

In [9]:
def toggle_safe_mode():

      msg ="\n\nTwo modes are possible for crawling.\n\n"\
           "\033[1mAutomatic:\033[0m in this mode, the program automatically explores "\
           "the web and collects images up to \nthe maximum number specified by the user, "\
           "in this mode, \033[1munethical\033[0m images may also be discovered.\n\n"\
           "\033[1mSafe:\033[0m This is a controlled mode, in which 35 images that have "\
           "already been collected are processed.\n"

      print(msg)

      print("Proceed safely?(\033[1mY/N\033[0m)")

      mode = input()
      
      return mode


<b>Implementation merge_bounding_boxes</b><br/>
The easyocr reader returns the output will be in a list format, each item represents a bounding box, the text detected and confident level, respectively.

In [10]:
def merge_bounding_boxes(result:list):
  merged=str('')
  for item in result:
    merged = merged + ' '+ item
  return merged

Implementation generate_html_table

In [20]:
def generate_html_table(processed_data=None,directory_name=None):

    print("\nGenerateing HTML table started ...")

    if processed_data == None:
         print("Stoped: No processed data found!")
         return
    
    count = len(processed_data)
    str_rows=""
    row_header = "<tr>\n"\
                 "\t<td align='center'><h1>Quotes</h1></td>\n"\
                 "\t<td align='center'><h1>Wallpapers</h1></td>\n"\
                 "\t<td align='center'><h1>Generated</h1></td>\n"\
                 "</tr>"

    for i in range(count):

          source = processed_data[i]["source_file"]
          # remove parent directory
          #source = source.replace(work_directory,'')

          wallpaper=processed_data[i]["wallpaper"]
          #wallpaper = wallpaper.replace(work_directory,"")

          generated=processed_data[i]["generated"]
          #generated = generated.replace(work_directory,'')
          row_str ="<tr>\n"\
                   "\t<td><img src='."+source+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+wallpaper+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+generated+"' width='300' height='200'/></td>\n"\
                   "</tr>"
          
          str_rows = str_rows + row_str
    
    print("Collected " +str(count)+" images ...")

    header_text = "<h1>Image crawler</h1><br/>\n" \
                  "<b>Abdullah Mohammadi<br/>abdhmohammady@gmail.com</b><br/><br/>\n"\
                  "The result of this table is collected based on crawling the web with the term 'AnneShirleyQuotes'.<br/>\n" \
                  "After collecting about 100 images with the theme of 'Anne Shirley's Quotes', we extracted the<br/>\n"\
                  "text of the images and placed the Persian translation on a new image.<br/><br/>\n"\
                  "In the table below:<br/>\n"\
                  "The 'Quotes' column from the search results of 'AnneShirleyQuotes',<br/>\n"\
                  "The 'Wallpapers' column from the search results of 'sunny' and<br/>\n"\
                  "The 'Generated' column are processed results.<br/><br/>\n"

    table_str = "<table align='center' border='1'>\n" + row_header + str_rows + "\n</table>"

    html_text = header_text + table_str
    
    #hs = open(directory_name+"/RESULTS.html", 'w')
    #hs.write(html_text)
    #hs.close() 
    try:
      readme = open(directory_name+"/README.MD", 'w')
      readme.write(html_text)
      readme.close() 

      print("HTML table saved in:", "/README.MD")
    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

      print()


Implementation of remove_folder_content

In [12]:
def remove_folder_content(folder=None):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

Implementation auto_crawl

In [16]:
#Valid toot path must be is 'workspace' or 'workspace' + 'safe_images/'
def auto_crawl(max_crawl = 20,quote_path=None,threshold=90, font = None):
  """
  Returns a dictiunary {
                        "undiscovered":string list of file names,
                        "ignored":{"match": float,"image_file":string},
                        "processed":list of {"source_file":string , "wallpaper":string , "generated":string}
                        }
  """
  # The path of the files from which no text is detected is saved here.
  undiscovered_files=[]
  # The path of the files whose discovered citation matches are less 
  # than the specified threshold is stored.
  ignored_img_list = []
  # The path of the files that were processed correctly is saved.
  processed_images = []

  # Get image names stored in anne shirley qoutes folder
  anne_img_names = get_crawled_image_names(quote_path+"/anne_quotes")
  
  # Get image names stored in wallpapers folder
  wallpaper_img_names = get_crawled_image_names(quote_path+"/wallpapers")
  
  image_count = min(len(anne_img_names), len(wallpaper_img_names))

  if image_count < max_crawl: max_crawl = image_count

  for i in range(max_crawl):

      anne_img_name      = quote_path + "/anne_quotes/" + anne_img_names[i]

      wallpaper_img_name = quote_path + "/wallpapers/" + wallpaper_img_names[i]

      generated_img_name = output_path +"/"+ str(i + 1)+".jpg"
      #
      #anne_img = Image.open(anne_img_name)
      # detect text from the image
      #text = pytesseract.image_to_string(image = anne_img , lang='eng')
      print(str(i+1)+" easyocr reads from: '"+ anne_img_name.replace(work_directory,"")+"'")
      easyocr_result = easyocr_reader.readtext(anne_img_name, detail = 0)
      # The output will be in a list format, each item represents a bounding box, the text detected and confident level, respectively.
      text = merge_bounding_boxes(easyocr_result)
      #print("detected text: ",text)
      # Cleaning text 
      text = clean_text(text = text)
      
      if text == "": 
        undiscovered_files.append(anne_img_name)
        print("No text discovered from :",anne_img_name.replace(work_directory,""))
        continue

      # Spell Cheching
      final_text = spell_check(text=text)
      
      # Step 3: Call fuzzy_match to check quote validation in anne books
      obtained_match,book = fuzzy_match(str1 = final_text,book_path= anne_book_path)

      # If matching ratio is less than 90% we ignore this quote and go to next quote
      if obtained_match < threshold :
        ignored_img_list.append({"match": obtained_match,"image_file":anne_img_names[i]})

        print()
        print("Ignored due to less than "+ str(threshold) +"% text match.\nfile name: "+anne_img_name.replace(work_directory,""))
        print("\033[1mThe obtained match " + str(obtained_match)+"%\033[0m")
        print()
        continue # go to next quote 
      # Step 3: Translate the text to Persian
      translated = translator.translate(text=final_text)

      #   End of cleaning text 
      #_____________________________________________________________________________________________
      
      # Step 4: Generate output image
      
      # Step 4.1: Open image from wallpapers
      out_img = Image.open(wallpaper_img_name)
      #_____________________________________________________________________________________________
      # Text placement cumputation 
    
      # Calculate the average length of a single character of our font.
      # Note: this takes into account the specific font and font size.
      avg_char_width = sum(font.getsize(char)[0] for char in ascii_letters) / len(ascii_letters)
      # Translate this average length into a character count
      # to fill 95% of our image's total width
      max_char_count = int( (out_img.size[0] * .95) / avg_char_width )

      # End of Text placement computation
      #_____________________________________________________________________________________________

      # Create a wrapped text object using scaled character count
      scaled_wrapped_text = textwrap.fill(text=translated, width = max_char_count)

      #print("\033[1mFrom Image "+str(i+1)+": \033[0m",text)
      #print("\033[1mTranslated: \033[0m",scaled_wrapped_text)
      #print()#empty line 
      
      # Step 4.2: Call draw Method to add 2D graphics in an image
      draw = ImageDraw.Draw(out_img)

      # Step 4.3: insert text to the Draw object

      #  4.3.1: make shape of text
      # correct its shape
      #reshaped_text = arabic_reshaper.reshape(translated)    
      
      #  4.3.2: make direction of text
      # correct its direction
      #bidi_text = get_display(reshaped_text)                            
      
      # 4.3.3 Calculate center point of text placement
      width  = out_img.width
      height = out_img.height
      img_center = (width/2,height/2)
      
      #  4.3.4: Draw text in image
      draw.text(xy = img_center,
                text = scaled_wrapped_text,
                fill="Black",
                stroke_fill="Gold",
                stroke_width=10,
                font=font,
                direction="rtl",
                language='fa',
                anchor ='mm',
                Literal='center',
                align='right')
      
      # Step 5: Save output in Generated-Image path
      out_img.save(generated_img_name)
      
      anne_img_name = anne_img_name.replace(work_directory,"")
      wallpaper_img_name = wallpaper_img_name.replace(work_directory,"")
      generated_img_name = generated_img_name.replace(work_directory,"")

      processed_images.append({"source_file":anne_img_name,
                               "wallpaper":wallpaper_img_name ,
                               "generated":generated_img_name})

      print("Generated image(text match:"+str(obtained_match)+"%) #",generated_img_name)
  
  return {"undiscovered":undiscovered_files,"ignored":ignored_img_list,"processed":processed_images}


<h1><b>Main body</b></h1>
This part is the main body of the code to perform image crawling,
Two modes are possible for crawling.<br/>

1. Automatic: In this mode, the program automatically explores the web. We can directly crawl and collect citations from the web and then process them. In this method, because we do <b>not monitor</b> the received images, there is a possibility of seeing <b>unethical images</b>. In this case, the number of images is not limited.

2. Safe: This is a <b>controlled</b> mode, We have already collected 100 images with 'icrawler' tools, among them we selected 35 images of quotes that have suitable background images, this collection is stored in path '/safe_images', in this case this collection is used for text discovery.

In both cases, an attempt is made to extract the text of the image. If no text is discovered, the further processing of that image is prevented, if it is found, it is matched with Anne Shirley's book content. If the matching percentage is higher than <b>specified threshold</b>, the work continues. Otherwise, this image is discarded, the default matching threshold is <b>90%</b>, which can be changed.
At the end, a README.MD file is created to show the result of the operation.

In [19]:

max_number = 35
threshold = 90
#Toggles Safe mode and Automatic Made
mode =  toggle_safe_mode()
# Automatic: in this mode, the program automatically explores the web and collects 
# images up to a maximum number specified in the 'Initalization' cell.
# in this mode,unethical images may also be discovered.
# Safe: This is a controlled mode, in which 35 images that have already been collected are processed.

n = datetime.datetime.now()

print ("Process started at %s:%s:%s" % (n.hour, n.minute, n.second))

print("Max number:", max_number)

print("Threshold:",threshold)

if mode not in ["Y","y","N","n"]:
    sys.exit('Terminate by wrong keywords!')

workspace = os.getcwd()
# Calling os.getcwd() has the following results:
# In google colab it returns '/content', This is the root directory.
# In GitHub codespace returns '/workspaces/codespaces-jupyter' 
# but this is the work directory.

# In GitHub codespace, our work directory is '/workspaces/codespaces-jupyter'
work_directory = workspace +""

# ENABLE THIS LINE IN 'Google Colab' WORKSPACE
# DISABLE THIS LINE IN GitHub' CODESPACE
# In google colab, our work directory is '/content/drive/MyDrive/Colab Notebooks/Text-Mining'
#work_directory = workspace + "/drive/MyDrive/Colab Notebooks/Text-Mining"

# When workspace and work directory was specified ...
font_path = work_directory+"/fonts"
anne_book_path = work_directory+"/anne_books"
# Directory to generate wallpapers from crawled images
output_path = work_directory+"/generated_images"

if not os.path.exists(output_path): os.makedirs(output_path)
else: remove_folder_content(output_path)

# Safe crawling is requested
if mode in ["Y","y"]:
   print("Safe mode selected.\n")
   # Defualt 'anne_quote' and 'wallpapers' folders located here.
   quote_path = work_directory + "/safe_images"
# Automatic crawling is requested, In automatic mode:
# In the first step, the images of Anne Shirley are searched
# In the second name step, the folder of the found images is renamed to the folder of 'anne_quotes.
# In the third step, wallpaper images are searched
# In the fourth step, the name of the folder of the found images is changed to that of 'wallpapers'.
elif mode in ["N","n"]:
   print("Automatic mode selected.\n")
   quote_path = workspace
   # When 'anne qoutes' crawled,by defualt,crawler creates 'images' folder in root directory
   # to manage root diregtory, we store it in 'workspace' varible.
   # Clear content of 'images' folder
   if os.path.exists(quote_path+"/images"):
      #remove_folder_content(workspace+"/images")
      # deletes the directory and all its contents.
      shutil.rmtree(quote_path +"/images")
   
   # Step 1: Crawling starts
   bing_crawler = bing.BingImageCrawler()
   bing_crawler.crawl(keyword = anne_keyword, offset = 0, max_num = max_number, min_size = None, max_size = None)
   #by default the crawler creates a folder as 'images' in root directory
   # and stores crawled images in, we rename this folder to 'anne_quotes'
   # becuase we want crawl wallpapers again.
    
   # Step 2: Rename crawled 'images' folder to 'anne_quotes'
   if os.path.exists(quote_path +"/anne_quotes"):
      #remove_folder_content(workspace +"/anne_quotes")
      shutil.rmtree(quote_path +"/anne_quotes")

   os.rename(quote_path+"/images",quote_path + "/anne_quotes")
   
   # Step 3: Wallpapers Crawling starts
   #google_crawler = google.GoogleImageCrawler()
   bing_crawler.crawl(keyword = wallpaper_keyword, offset = 0, max_num = max_number, min_size = None, max_size = None)
   #by default the crawler creates a folder as 'images' in root directory
   # and stores crawled images in, we rename this folder to 'wallpapers'
   
   # Step 4: Rename crawled 'images' folder to 'wallpapers'
   if os.path.exists(quote_path +"/wallpapers"):
      shutil.rmtree(quote_path +"/wallpapers")
   
   if os.path.exists(quote_path+"/images"):
      os.rename(quote_path+"/images",quote_path +"/wallpapers")

font_ = ImageFont.truetype(font = font_path +'/Vazirmatn-Regular.ttf',size= 70)

result = auto_crawl(max_crawl= max_number , quote_path = quote_path , threshold=threshold, font = font_)

generate_html_table(processed_data = result["processed"],directory_name = work_directory)

processed = len(result["processed"])

ignored = len(result["ignored"])

undiscovered = len(result["undiscovered"])

print()
print("-------------------------------------------------------------------------------------")
print("\033[1mText matching threshold\033[0m:",threshold)
print("\033[1mProcessed images\033[0m:",processed)
print("\033[1mIgnored images\033[0m:",ignored)

for item in result["ignored"]:
    print(item["image_file"]+", obtained ratio : ",str(item["match"])+"%")

print("\033[1mUndiscovered images\033[0m : ",undiscovered)

for item in result["undiscovered"]:
       print(item)
n = datetime.datetime.now()
print ("All tasks done at %s:%s:%s" % (n.hour, n.minute, n.second))

Two modes are possible for crawling.

Automatic: in this mode, the program automatically explores the web and collects images up to 
a maximum number specified by the user, in this mode, unethical images may also be discovered.

Safe: This is a controlled mode, in which 35 images that have already been collected are processed.

Proceed safely?(Y/N)
y
Process started at 8:36:55
Max number: 35
Threshold: 90
Safe mode selected.

easyocr reads from: '/safe_images/anne_quotes/32.jpg'
Generated image(text match:93%) # /generated_images/1.jpg
easyocr reads from: '/safe_images/anne_quotes/34.jpg'
Generated image(text match:100%) # /generated_images/2.jpg
easyocr reads from: '/safe_images/anne_quotes/11.jpg'
Generated image(text match:93%) # /generated_images/3.jpg
easyocr reads from: '/safe_images/anne_quotes/12.jpg'
Generated image(text match:94%) # /generated_images/4.jpg
easyocr reads from: '/safe_images/anne_quotes/26.jpg'
Generated image(text match:96%) # /generated_images/5.jpg
easyocr r